# Import Libraries

In [1]:
import pandas as pd
import numpy as np

import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

## importing plotly 
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [3]:
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

In [4]:
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

# Loading TSV's with Pandas

In [5]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [6]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [7]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

# Preprocessing Title Basics

## Handling \N Placeholder Values

In [8]:
basics.replace({'\\N':np.nan}, inplace = True)

In [9]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


## Eliminate movies that are null for runtimeMinutes

In [10]:
# Check for duplicates
basics.duplicated().sum()

0

In [11]:
# Check for null values
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1358165
endYear           9994745
runtimeMinutes    7084595
genres             454924
dtype: int64

In [12]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)

In [13]:
basics['runtimeMinutes'].isna().sum()

0

## Eliminate movies that are null for genre

In [14]:
# Eliminate movies that are null for genres
basics.dropna(subset=['genres'], inplace=True)

In [15]:
# Null values in genres has been eliminated
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear          173741
endYear           2890327
runtimeMinutes          0
genres                  0
dtype: int64

## Keep only titleType==Movie

In [16]:
#Keep only titleType equal to Movie
basics['titleType'].value_counts()

tvEpisode       1528683
short            611357
movie            387781
video            184261
tvMovie           92617
tvSeries          91708
tvSpecial         18726
tvMiniSeries      17829
tvShort            8716
videoGame           332
Name: titleType, dtype: int64

In [17]:
basics = basics.drop(basics[basics['titleType'] != 'movie'].index)

In [18]:
basics['titleType'].value_counts()

movie    387781
Name: titleType, dtype: int64

## Keep startYear 2000-2021

In [19]:
# Keep startYear 2000-2022
basics['startYear'].value_counts().sort_index()

1894      1
1896      1
1897      1
1899      1
1900      2
       ... 
2024    202
2025     31
2026      8
2027      1
2029      2
Name: startYear, Length: 131, dtype: int64

In [20]:
basics['startYear'] = basics['startYear'].astype('float')

In [21]:
year_range = (basics['startYear'] >= 2000) & (basics['startYear'] <= 2021)
basics = basics[year_range]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13081,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021.0,NaN,94,Documentary
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61112,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67486,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67664,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama


In [22]:
basics['startYear'].value_counts()

2017.0    14425
2018.0    14408
2019.0    14151
2016.0    14003
2015.0    13512
2014.0    13159
2021.0    12496
2013.0    12418
2012.0    11675
2020.0    11648
2011.0    10796
2010.0    10233
2009.0     9384
2008.0     8185
2007.0     6985
2006.0     6555
2005.0     5862
2004.0     5228
2003.0     4620
2002.0     4152
2001.0     3894
2000.0     3659
Name: startYear, dtype: int64

## Eliminate movies that include "Documentary" in genre

In [23]:
# Eliminate movies that include "Documentary" in genre
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

# Preprocessing AKAs

## Replace "\N" with np.nan

In [24]:
akas.replace({'\\N':np.nan}, inplace = True)

In [25]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


## Keep only US movies

In [26]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [27]:
us_only = akas['region'] == 'US'

In [28]:
akas = akas[us_only]
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0
...,...,...,...,...,...,...,...,...
36961964,tt9916560,1,March of Dimes Presents: Once Upon a Dime,US,NaN,imdbDisplay,NaN,0
36962034,tt9916620,1,The Copeland Case,US,NaN,imdbDisplay,NaN,0
36962123,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0
36962166,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0


# Basics Data - Keep only US Movies

In [29]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34800        True
61112        True
67486        True
67664        True
86791        True
            ...  
10106022     True
10106061    False
10106106     True
10106190    False
10106280    False
Name: tconst, Length: 139019, dtype: bool

In [30]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61112,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67486,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67664,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86791,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
10105479,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
10105873,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
10106013,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
10106022,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


# Preprocessing Ratings

In [31]:
ratings.replace({'\\N':np.nan}, inplace = True)

In [32]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1989
1,tt0000002,5.8,264
2,tt0000003,6.5,1863
3,tt0000004,5.5,177
4,tt0000005,6.2,2648


In [33]:
# Filter the ratings table down to only include the US movies
ratings_us =ratings['tconst'].isin(akas['titleId'])

In [34]:
ratings = ratings[ratings_us]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1989
1,tt0000002,5.8,264
4,tt0000005,6.2,2648
5,tt0000006,5.0,182
6,tt0000007,5.4,829
...,...,...,...
1341621,tt9916200,8.1,233
1341622,tt9916204,8.1,267
1341629,tt9916348,8.3,18
1341630,tt9916362,6.4,5494


# Final .info() 

In [35]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82012 entries, 34800 to 10106106
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82012 non-null  object 
 1   titleType       82012 non-null  object 
 2   primaryTitle    82012 non-null  object 
 3   originalTitle   82012 non-null  object 
 4   isAdult         82012 non-null  object 
 5   startYear       82012 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  82012 non-null  object 
 8   genres          82012 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.3+ MB


In [36]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506634 entries, 0 to 1341635
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         506634 non-null  object 
 1   averageRating  506634 non-null  float64
 2   numVotes       506634 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.5+ MB


In [37]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461694 entries, 5 to 36962182
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1461694 non-null  object
 1   ordering         1461694 non-null  int64 
 2   title            1461694 non-null  object
 3   region           1461694 non-null  object
 4   language         4114 non-null     object
 5   types            983053 non-null   object
 6   attributes       47354 non-null    object
 7   isOriginalTitle  1460352 non-null  object
dtypes: int64(1), object(7)
memory usage: 100.4+ MB


# Creating a "Data" folder

In [38]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

[]

In [39]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [40]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [41]:
## Save current dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [42]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [43]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1989
1,tt0000002,5.8,264
2,tt0000005,6.2,2648
3,tt0000006,5.0,182
4,tt0000007,5.4,829


In [44]:
# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0
